# Sending event data in parallel directly into QuestDB

*Note*: For an overview of sending data directly into QuestDB, please see first the `IoTEventsToQuestDB` notebook.

Even if QuestDB can ingest several millions of events per second, it is unlikely a single process can send as much data over a single connection. In order to get multi-million per second rates, real-life scenarios ingest data from a few different clients and a few different connections to the database.


This script simulates sending data in parallel from 6 different processes.

The notebook connects to port `9000` of the `questdb` container and sends data continuously (the client library will automatically flush every 75K rows by default). You can configure a delay between rows if you want by changing the the `BATCH` and `DELAY` constants. 

Take into account that this will write potentially a lot of data in the `./questdb/questdb_root` folder at your repository root. It is safe to remove the contents (but please keep the folder) if you want to free disk space.

A live dashboard displaying this dataset can be seen at [http://localhost:3000/d/qdb-iot-parallel-demo/parallel-data-demo?orgId=1&refresh=1s](http://localhost:3000/d/qdb-iot-parallel-demo/parallel-data-demo?orgId=1&refresh=1s) . User is `admin` and password `quest`.


In [7]:
from questdb.ingress import Sender, IngressError, TimestampNanos
import os
import sys
import random
import time
from multiprocessing import Pool
import math

HTTP_ENDPOINT = os.getenv('QUESTDB_HTTP_ENDPOINT', 'questdb:9000')
REST_TOKEN = os.getenv('QUESTDB_REST_TOKEN')

DEVICE_TYPES = ["blue", "red", "green", "yellow"]
TOTAL_EVENTS = 1_000_000  # Total events across all senders
BATCH = 75000  # Used for sleep intervals
DELAY = 10 
NUM_SENDERS = 7  # Number of senders to execute in parallel

def send(sender_id, total_events, batch=None, http_endpoint: str = None, auth=None):
    sys.stdout.write(f"Sender {sender_id} will send {total_events} events\n")

    try:
        if auth is not None:
            conf = f'https::addr={http_endpoint};tls_verify=unsafe_off;token={auth};'
        else:
            conf = f'http::addr={http_endpoint};'

        sys.stdout.write(conf)
        with Sender.from_conf(conf) as sender:
            events_sent = 0
            while events_sent < total_events:
                current_batch_size = min(batch, total_events - events_sent)
                for _ in range(current_batch_size):
                    sender.row(
                        'iot_data',
                        symbols={'device_type': random.choice(DEVICE_TYPES)},
                        columns={
                            'duration_ms': random.randint(0, 4000),
                            "measure1": random.randint(-2147483648, 2147483647),
                            "measure2": random.randint(-2147483648, 2147483647),
                            "speed": random.randint(0, 100)
                        },
                        at=TimestampNanos.now())
                events_sent += current_batch_size
                time.sleep(DELAY)
            sys.stdout.write(f"Sender {sender_id} finished sending {events_sent} events\n")
    except IngressError as e:
        sys.stderr.write(f'Sender {sender_id} got error: {e}\n')

def parallel_send(total_events, num_senders: int, batch, http_endpoint, auth):
    events_per_sender = math.floor(total_events / num_senders)
    remaining_events = total_events % num_senders

    sender_events = [events_per_sender] * num_senders
    for i in range(remaining_events):  # distribute the remaining events
        sender_events[i] += 1

    with Pool(processes=num_senders) as pool:
        sender_ids = range(num_senders)
        pool.starmap(send, [(sender_id, sender_events[sender_id], batch, http_endpoint, auth) for sender_id in sender_ids])

if __name__ == '__main__':
    sys.stdout.write(f'Ingestion started. Connecting to {HTTP_ENDPOINT}\n')
    parallel_send(TOTAL_EVENTS, NUM_SENDERS, BATCH, HTTP_ENDPOINT, REST_TOKEN)


Ingestion started. Connecting to 172.31.42.41:9000
Sender 1 will send 142857 events
Sender 2 will send 142857 events
Sender 3 will send 142857 events
Sender 0 will send 142858 events
Sender 4 will send 142857 events
Sender 5 will send 142857 events
Sender 6 will send 142857 events
https::addr=172.31.42.41:9000;tls_verify=unsafe_off;token=qt1H8Mi_vPUpt_R7ByDdJ7_kRIiieNb_BFKdjBDqxp6wh8;https::addr=172.31.42.41:9000;tls_verify=unsafe_off;token=qt1H8Mi_vPUpt_R7ByDdJ7_kRIiieNb_BFKdjBDqxp6wh8;https::addr=172.31.42.41:9000;tls_verify=unsafe_off;token=qt1H8Mi_vPUpt_R7ByDdJ7_kRIiieNb_BFKdjBDqxp6wh8;https::addr=172.31.42.41:9000;tls_verify=unsafe_off;token=qt1H8Mi_vPUpt_R7ByDdJ7_kRIiieNb_BFKdjBDqxp6wh8;https::addr=172.31.42.41:9000;tls_verify=unsafe_off;token=qt1H8Mi_vPUpt_R7ByDdJ7_kRIiieNb_BFKdjBDqxp6wh8;https::addr=172.31.42.41:9000;tls_verify=unsafe_off;token=qt1H8Mi_vPUpt_R7ByDdJ7_kRIiieNb_BFKdjBDqxp6wh8;https::addr=172.31.42.41:9000;tls_verify=unsafe_off;token=qt1H8Mi_vPUpt_R7ByDdJ7_kRIiieN